In [0]:
import pyspark.sql.functions as F

india_df = spark.table('hive_metastore.azure_sql_db_dbo.crimes_on_women')
india_df.display()

_fivetran_id,dd,rape,dv,year,state,ka,aow,id,wt,aom,_fivetran_deleted,_fivetran_synced
0717E9B95BB0D9A2,0,1,4,2017,D&N Haveli,0,3,587,0,0,false,2025-01-25T02:58:06.434Z
07FB535288990BDC,60,1095,1808,2003,ASSAM,1351,878,70,0,6,false,2025-01-25T02:58:06.415Z
0D18037FE78B9DBE,0,5,6,2014,D&N Haveli,7,2,479,0,0,false,2025-01-25T02:58:06.432Z
0C6732E11F444FB8,278,1109,1002,2016,Jharkhand,693,667,530,0,373,false,2025-01-25T02:58:06.433Z
0EF62A18F763D27B,0,75,21,2021,Manipur,42,92,715,3,24,false,2025-01-25T02:58:06.436Z
02E3CFDEF26FC659,0,6,15,2016,Puducherry,11,43,555,0,13,false,2025-01-25T02:58:06.433Z
085AB4BAF4DF8A49,1,4,2,2020,Mizoram,6,7,681,0,2,false,2025-01-25T02:58:06.435Z
0E3D9181C391CA79,130,442,801,2006,PUNJAB,418,314,190,2,60,false,2025-01-25T02:58:06.423Z
0F6812CC09E7ECBA,53,841,5991,2014,Gujarat,2187,1352,454,45,405,false,2025-01-25T02:58:06.431Z
0A84CD115D685614,1,26,3,2016,Nagaland,50,14,538,0,6,false,2025-01-25T02:58:06.433Z


column names are not clear, luckily the dataset includes a data description table
|      |   Column Names  |   Explanation                       |
|------|-----------------|-------------------------------------|
|   0  |   State         |   State                             |
|   1  |   Year          |   Year                              |
|   2  |   Rape          |   No. of Rape cases                 |
|   3  |   K&A           |   Kidnap And Assault                |
|   4  |   DD            |   Dowry Deaths                      |
|   5  |   AoW           |   Assault against women             |
|   6  |   AoM           |   Assault against modesty of women  |
|   7  |   DV            |   Domestic violence                 |
|   8  |   WT            |   Women Trafficking                 |

In [0]:
india_df_renamed = india_df.where(
    F.col('_fivetran_deleted') == False,
).withColumn(
    'country', F.lit('india'),
).select(
    F.col('country'),
    F.col('Year').alias('year'),
    F.col('State').alias('state'),
    F.col('Rape').alias('rape'),
    F.col('ka').alias('kidnap_assault'),
    F.col('dd').alias('dowry_deaths'),
    F.col('dv').alias('domestic_violence'),
    F.col('aow').alias('assault'),
    F.col('aom').alias('assault_on_modesty'),
)
india_df_renamed.display()

country,year,state,rape,kidnap_assault,dowry_deaths,domestic_violence,assault,assault_on_modesty
india,2017,D&N Haveli,1,0,0,4,3,0
india,2003,ASSAM,1095,1351,60,1808,878,6
india,2014,D&N Haveli,5,7,0,6,2,0
india,2016,Jharkhand,1109,693,278,1002,667,373
india,2021,Manipur,75,42,0,21,92,24
india,2016,Puducherry,6,11,0,15,43,13
india,2020,Mizoram,4,6,1,2,7,2
india,2006,PUNJAB,442,418,130,801,314,60
india,2014,Gujarat,841,2187,53,5991,1352,405
india,2016,Nagaland,26,50,1,3,14,6


now we melt the table

In [0]:
india_df_melted = india_df_renamed.melt(
    ids=['country', 'year', 'state'],
    values=None,
    valueColumnName='total_cases',
    variableColumnName='type_of_violence',
)
india_df_melted.display()

country,year,state,type_of_violence,total_cases
india,2017,D&N Haveli,rape,1
india,2017,D&N Haveli,kidnap_assault,0
india,2017,D&N Haveli,dowry_deaths,0
india,2017,D&N Haveli,domestic_violence,4
india,2017,D&N Haveli,assault,3
india,2017,D&N Haveli,assault_on_modesty,0
india,2003,ASSAM,rape,1095
india,2003,ASSAM,kidnap_assault,1351
india,2003,ASSAM,dowry_deaths,60
india,2003,ASSAM,domestic_violence,1808


sum all violence cases and yearly total

In [0]:
india_df_any_violence = india_df_melted.groupBy('year', 'country', 'state', 'type_of_violence').agg(
    F.sum('total_cases').alias('total_cases')
).withColumn(
    'type_of_violence', F.lit('any')
).select(
    F.col('country'),
    F.col('year'),
    F.col('state'),
    F.col('type_of_violence'),
    F.col('total_cases')
)
india_df_any_violence = india_df_melted.union(india_df_any_violence)
india_df_total = india_df_any_violence.groupBy('year', 'country', 'type_of_violence').agg(
    F.sum('total_cases').alias('total_cases')
).withColumn(
    'state', F.lit('all')
).select(
    F.col('country'),
    F.col('year'),
    F.col('state'),
    F.col('type_of_violence'),
    F.col('total_cases')
)
india_df_total = india_df_total.union(india_df_any_violence)
india_df_total.display()

country,year,state,type_of_violence,total_cases
india,2007,all,assault,37866
india,2009,all,kidnap_assault,24086
india,2013,all,rape,33707
india,2010,all,assault,40012
india,2013,all,assault_on_modesty,12589
india,2015,all,assault,82422
india,2002,all,rape,15970
india,2015,all,rape,34651
india,2007,all,dowry_deaths,7955
india,2019,all,domestic_violence,125298


now we need to delete the '_' character form the year